In [ ]:
# data preprocessing

In [56]:
import trl

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer,SFTConfig

if torch.cuda.is_available():
    device = torch.device("cuda")
    #print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("CUDA not available, using CPU.")

C:\Users\z's'q\AppData\Local\Temp\ipykernel_1568\2964636124.py:4: FutureWarning: Support for Python 3.9 will be dropped in the next release (after its end-of-life on October 31, 2025). Please upgrade to Python 3.10 or newer.
  from trl import SFTTrainer,SFTConfig


In [97]:
import json
with open("datasets3.json", "r", encoding="utf-8") as f:
    geography_datasets = json.load(f)  

In [98]:
geography_datasets["train"] = [
    {"prompt":item["question"], "completion":item["sql"]}
    for item in geography_datasets["train"]
]

geography_datasets["dev"] = [
    {"prompt":item["question"], "completion":item["sql"]}
    for item in geography_datasets["dev"]
]

geography_datasets["test"] = [
    {"prompt":item["question"], "completion":item["sql"]}
    for item in geography_datasets["test"]
]

# Augmented data

In [3]:
import json
with open("newdatasets.json", "r", encoding="utf-8") as f:
    geography_datasets = json.load(f)  

In [4]:
from datasets import Dataset,DatasetDict

train_data =  Dataset.from_list(geography_datasets["train"])
dev_data =  Dataset.from_list(geography_datasets["dev"])
test_data =  Dataset.from_list(geography_datasets["test"])
dataset = DatasetDict({"train": train_data, "dev": dev_data, "test": test_data})

# model training

In [11]:
# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-360M-Instruct"  
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Check and add only the truly new tokens
new_tokens_to_add = set(new_tokens) - set(tokenizer.vocab.keys())
if new_tokens_to_add:
    tokenizer.add_tokens(list(new_tokens_to_add))
    print(f"Added {len(new_tokens_to_add)} new tokens.")
else:
    print("No new tokens to add.")
# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token # Use eos_token as pad_token
model.resize_token_embeddings(len(tokenizer))

#write your prompts and try to evaluate the model
making_prompt = lambda x:x

Added 169 new tokens.


In [12]:
# try figure out how to set up the fine tuning config
sft_config= SFTConfig(learning_rate=0.001,
    torch_empty_cache_steps=3,
    num_train_epochs=2,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size=2,
    weight_decay = 0.001,
    logging_steps=20,
    completion_only_loss=True
    )


model.to(device)
# Create the SFT Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["dev"],
    args=sft_config,
    processing_class = tokenizer # Pass the tokenizer
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_model('models/fine_tuned_model2')
tokenizer.save_pretrained('models/fine_tuned_model2') 

Adding EOS to train dataset:   0%|          | 0/749 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/749 [00:00<?, ? examples/s]

Mismatch between tokenized prompt and the start of tokenized prompt+completion. This may be due to unexpected tokenizer behavior, whitespace issues, or special token handling. Verify that the tokenizer is processing text consistently.
Mismatch between tokenized prompt and the start of tokenized prompt+completion. This may be due to unexpected tokenizer behavior, whitespace issues, or special token handling. Verify that the tokenizer is processing text consistently.


Truncating train dataset:   0%|          | 0/749 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/749 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/749 [00:00<?, ? examples/s]

Mismatch between tokenized prompt and the start of tokenized prompt+completion. This may be due to unexpected tokenizer behavior, whitespace issues, or special token handling. Verify that the tokenizer is processing text consistently.
Mismatch between tokenized prompt and the start of tokenized prompt+completion. This may be due to unexpected tokenizer behavior, whitespace issues, or special token handling. Verify that the tokenizer is processing text consistently.


Truncating eval dataset:   0%|          | 0/749 [00:00<?, ? examples/s]

Step,Training Loss
20,4.158600
40,1.528000
60,1.081700
80,0.889800
100,0.817500
120,0.820500
140,0.531800
160,0.630800
180,0.437900
200,0.485600


('models/fine_tuned_model2\\tokenizer_config.json',
 'models/fine_tuned_model2\\special_tokens_map.json',
 'models/fine_tuned_model2\\chat_template.jinja',
 'models/fine_tuned_model2\\vocab.json',
 'models/fine_tuned_model2\\merges.txt',
 'models/fine_tuned_model2\\added_tokens.json',
 'models/fine_tuned_model2\\tokenizer.json')

In [13]:
torch.cuda.empty_cache() 

In [14]:
# Prepare prompt
prompt = "what is the biggest city of wyoming"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        do_sample=True,
        temperature=0.9,
        top_p=0.98,
        pad_token_id=tokenizer.eos_token_id
    )

# Decode and print only the new response
response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
print("Assistant:", response.strip())

C:\anaconda3\lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Assistant: SELECT CITYalias0 = "" ; largestSELECT CITYalias0 = " " " " "" = "" > > ; outsideSELECT 1 instantaneous CITYalias0. "" > > > > > ; highestSELECT 1 reversible = "" ; supposed combined CITYalias0 = "" "" ; surroundingSELECT BORDER_INFO = " " " "" CO = " " " " "" longerSELECT island" > CITYalias0 = " " "" > (CITYalias1 = "" > ;}ushima = "" ;______FI = " "" ;. CITYalias0 = "" = "" OR = "


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
checkpoint = "HuggingFaceTB/SmolLM2-360M-Instruct"

device = "cuda" # for GPU usage or "cpu" for CPU usage
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# for multiple GPUs install accelerate and do `model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")`
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

messages = [{"role": "user", "content": "What is the capital of France."}]
input_text=tokenizer.apply_chat_template(messages, tokenize=False)
print(input_text)
inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=50, temperature=0.8, top_p=0.9, do_sample=True)
print(tokenizer.decode(outputs[0]))

# tokenizer

In [2]:
new_tokens = ['BORDER_INFO','STATE_NAME','BORDER','CITY','CITY_NAME','STATE_NAME','POPULATION','COUNTRY_NAME','HIGHLOW','STATE_NAME','HIGHEST_POINT','HIGHEST_ELEVATION','LOWEST_POINT',
 'LOWEST_ELEVATION','LAKE','LAKE_NAME','AREA','STATE_NAME','COUNTRY_NAME','MOUNTAIN','MOUNTAIN_NAME','MOUNTAIN_ALTITUDE','STATE_NAME','COUNTRY_NAME','RIVER','RIVER_NAME',
 'LENGTH','TRAVERSE','COUNTRY_NAME','ROAD','ROAD_NAME','STATE_NAME','STATE','STATE_NAME','CAPITAL','POPULATION','AREA','COUNTRY_NAME','DENSITY','SELECT',
 'CITYalias0','CITYNAME','FROM','AS','WHERE','MAX','CITYalias1','STATENAME','arizona','AND','texas','missouri','kansas','louisiana','california','rhode',
 'island','new','mexico','nebraska','wyoming','michigan','minnesota','alabama','oregon','georgia','wisconsin','RIVERalias0','RIVERNAME','IN','STATEalias0',
 'alaska','florida','ohio','maryland','maine','south','carolina','idaho','washington','north','dakota','massachusetts','mississippi','utah','hawaii',
 'york','nevada','illinois','montana','iowa','hampshire','MIN','STATEalias1','virginia','pennsylvania','albany','LAKEalias0','LAKENAME','750','colorado',
 'delaware','chattahoochee','potomac','red','HIGHLOWalias0','LOWESTELEVATION','HIGHESTPOINT','0','DISTINCT','RIVERalias1','COUNT','BORDERINFOalias0',
'BORDERINFO','indiana','jersey','kentucky','arkansas','west','DERIVEDTABLEalias0','DERIVEDFIELDalias0','GROUP','BY','dallas','san','diego','antonio',
 'austin','miami','plano','portland','rochester','salt','lake','city','kalamazoo','pittsburgh','flint','springfield','jose','scotts','valley',
 'orleans','indianapolis','des','moines','boston','fort','wayne','houston','baton','rouge','denver','chicago','detroit','boulder','tucson','francisco',
 'sacramento','seattle','montgomery','riverside','atlanta','HIGHESTELEVATION','HIGHLOWalias1','STATEalias2','STATEalias3','ORDER','DESC','LIMIT',
 '1','DERIVEDTABLEalias1','BORDERINFOalias1','HAVING','DERIVEDFIELDalias1','BORDERINFOalias2','NOT','mount','mckinley','guadalupe','peak',
 'platte','rio','grande','150000','dc','minneapolis','erie','tempe','spokane','SUM','tennessee','vermont','oklahoma','durham','MOUNTAINalias0',
 'MOUNTAINNAME','MOUNTAINALTITUDE','MOUNTAINalias1','RIVERalias2','LOWESTPOINT','COUNTRYNAME','usa','AVG','whitney','dover','columbus','salem',
 'STATEalias4','STATEalias5','LEFT','OUTER','JOIN','ON','RIVERalias3','ALL','death','BORDERINFOalias3']

In [ ]:
# Check and add only the truly new tokens
new_tokens_to_add = set(new_tokens) - set(tokenizer.vocab.keys())
if new_tokens_to_add:
    tokenizer.add_tokens(list(new_tokens_to_add))
    print(f"Added {len(new_tokens_to_add)} new tokens.")
else:
    print("No new tokens to add.")
# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token # Use eos_token as pad_token
model.resize_token_embeddings(len(tokenizer))